### URL To Image

In [3]:
from urllib.request import urlopen
from PIL import Image
from pytesseract import pytesseract
import numpy as np
import pandas as pd
import cv2
import re
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)
    return image

### Data Extraction from Image

In [4]:
#Define path to tessaract.exe
path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
pytesseract.tesseract_cmd = path_to_tesseract

#Define path to image
path_to_image = 'IR-M-S-8890.jpg'
path_to_image1 = 'IR-E-U-0456.jpg'
path_to_image2 = 'IR-E-U-0037.jpg'
def image_to_data(path_to_image):
#     image = cv2.imread(path_to_image)
    image=url_to_image(path_to_image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    invert = 255 - thresh
    text = pytesseract.image_to_string(invert, lang='eng', config='--psm 6')
#     print(text)
    arr=[]
    for line in text.splitlines():
        arr.append(line)
    arr=np.array(arr[-5:])
    x = arr[3].split(" ")
    
    if(len(x) != 19):
        x = arr[2].split(" ")
#     print(x)
#     if(x[1]=='SS' or x[1]=='ss'):
#         arr=np.array(arr[-5:])
#         x=arr[3].split(" ")
#     print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
#     print(arr[-3])
#     print(x)
    a=np.array(x)
    a=a[2:]
#     print(a)
    temp=[]
    flag=False
    for i in a:
        f=i.replace(",","")
        f=re.sub("[^0-9]", "", f)
        temp.append(f)
#         print(f)
        if(float(f)>100):
            flag=True
    a=np.array(temp)
    
#     print("******************************")
#     print(a)
    a = a.astype(float)
    if(flag):
        for i in range(len(a)):
            a[i]/=100
#     temp=[]
    final=a
#     for i in final:
#         j=float(i)
#         temp.append(j)
#     final=np.array(temp)
#     print(final)
    return final

# a=image_to_data(path_to_image)
b=image_to_data('https://www.nirfindia.org/nirfpdfcdn/2021/graph/Engineering/IR-E-U-0456.jpg')
# c=image_to_data(path_to_image2)

### Web Scrapping

In [5]:
from autoscraper import AutoScraper
url="https://www.nirfindia.org/2021/EngineeringRanking.html"
wanted_list=["IR-E-U-0456","Indian Institute of Technology Madras","https://www.nirfindia.org/nirfpdfcdn/2021/graph/Engineering/IR-E-U-0456.jpg","95.47","96.43","81.92","62.44","100.00","https://www.nirfindia.org/nirfpdfcdn/2021/pdf/Engineering/IR-E-U-0456.pdf","90.19"]
scraper=AutoScraper()
result=scraper.build(url,wanted_list)
rule=scraper.get_result_similar(url,grouped=True)
keys=list(rule.keys())
scraper.set_rule_aliases({keys[0]:'Institute ID',keys[1]:'Name',keys[2]:'image',keys[3]:'TLR',keys[4]:'RPC',keys[5]:'GO',keys[6]:'OI',keys[7]:'PERCEPTION',keys[8]:'pdf',keys[9]:'Score'})
scraper.keep_rules([keys[0],keys[1],keys[2],keys[3],keys[4],keys[5],keys[6],keys[7],keys[8],keys[9]])
scraper.save('nirf')

In [6]:
result=scraper.get_result_similar(url,group_by_alias=True)
# (result['Name'])

In [7]:
university_2021=scraper.get_result_similar('https://www.nirfindia.org/2021/UniversityRanking.html',group_by_alias=True)
len(university_2021['Score'])

100

In [8]:
university_2021=scraper.get_result_similar('https://www.nirfindia.org/2021/ManagementRanking.html',group_by_alias=True)
len(university_2021['OI'])

75

In [9]:
management_2020=scraper.get_result_similar('https://www.nirfindia.org/2020/ManagementRanking.html',group_by_alias=True)
# (management_2020['Score'])

In [10]:
architecture_2019=scraper.get_result_similar('https://www.nirfindia.org/2019/ArchitectureRanking.html',group_by_alias=True)
len(architecture_2019['TLR'])

15

In [11]:
engineering_2017=scraper.get_result_similar('https://www.nirfindia.org/2017/EngineeringRanking.html',group_by_alias=True)
len(engineering_2017['pdf'])

100

In [12]:
engineering_2021=scraper.get_result_similar('https://www.nirfindia.org/2021/EngineeringRanking.html',group_by_alias=True)
len(engineering_2021['pdf'])

200

In [13]:
array=[engineering_2021['Name'],engineering_2021['image'],engineering_2021['pdf']]
link=pd.DataFrame(array);
link=link.T
link.columns=['Name','Image','Pdf']
link.head()

,Name,Image,Pdf
0,Indian Institute of Technology Madras,https://www.nirfindia.org/nirfpdfcdn/2021/grap...,https://www.nirfindia.org/nirfpdfcdn/2021/pdf/...
1,Indian Institute of Technology Delhi,https://www.nirfindia.org/nirfpdfcdn/2021/grap...,https://www.nirfindia.org/nirfpdfcdn/2021/pdf/...
2,Indian Institute of Technology Bombay,https://www.nirfindia.org/nirfpdfcdn/2021/grap...,https://www.nirfindia.org/nirfpdfcdn/2021/pdf/...
3,Indian Institute of Technology Kanpur,https://www.nirfindia.org/nirfpdfcdn/2021/grap...,https://www.nirfindia.org/nirfpdfcdn/2021/pdf/...
4,Indian Institute of Technology Kharagpur,https://www.nirfindia.org/nirfpdfcdn/2021/grap...,https://www.nirfindia.org/nirfpdfcdn/2021/pdf/...


# Image Data

In [83]:
Parameters=['SS','FSR','FQE','FRU','PU','QP','IPR','FPPP','GPH','GUE','MS','GPHD','RD','WD','ESCS','PCS','PR']
array=[]
# image_to_data(df.loc[46,'Image'])
link.loc[46,'Name']
for i in range(link.shape[0]):
    try:
        temp=image_to_data(link.loc[i,'Image'])
        array.append([link.loc[i,'Name'],temp])
    except Exception:
        print(i)
        pass
print(len(array))

46
131
198


In [84]:
new_array=[]
temp={'Name':[]}
# print(len(array[0][1]))
# print(len(Parameters))
for i in range(len(array)):
    temp['Name'].append(array[i][0])
    if(len(array[i][1])==0):
        print(array[i][0])
new_array.append(temp)
for i in range(len(Parameters)):
    temp={Parameters[i]:[]}
    for j in range(len(array)):
        temp[Parameters[i]].append(array[j][1][i])
    new_array.append(temp)
print(new_array[1])

{'SS': [18.5, 18.5, 18.5, 18.25, 18.44, 18.11, 16.69, 10.32, 16.0, 16.0, 15.34, 18.54, 9.11, 17.51, 10.28, 17.09, 17.08, 18.02, 11.24, 17.21, 9.43, 10.27, 16.0, 16.69, 16.0, 17.87, 13.77, 9.13, 12.86, 14.38, 14.77, 17.32, 10.5, 17.44, 12.0, 16.63, 12.71, 16.07, 19.0, 6.95, 9.64, 14.2, 10.32, 13.34, 15.0, 10.17, 12.79, 13.82, 16.75, 15.5, 11.5, 15.5, 8.38, 14.91, 12.16, 6.94, 4.31, 7.27, 13.34, 17.0, 14.48, 10.54, 13.68, 12.33, 17.89, 12.07, 11.5, 10.14, 11.56, 11.5, 11.17, 5.63, 11.0, 10.89, 5.97, 14.0, 12.0, 7.28, 8.35, 5.02, 12.5, 13.96, 12.18, 5.88, 11.49, 9.04, 13.48, 10.72, 13.26, 11.98, 10.39, 12.02, 12.64, 11.5, 12.11, 7.28, 13.7, 11.59, 10.2, 9.84, 11.26, 7.72, 9.5, 11.38, 5.75, 8.16, 7.56, 12.58, 14.05, 13.55, 8.83, 13.82, 6.62, 4.43, 13.72, 14.0, 7.59, 10.5, 4.19, 13.29, 14.5, 8.38, 10.96, 7.71, 8.89, 9.5, 10.57, 7.63, 10.05, 7.5, 12.0, 10.4, 13.5, 6.32, 10.62, 12.0, 8.79, 7.5, 13.5, 10.5, 8.5, 9.68, 6.78, 13.5, 13.5, 6.19, 12.0, 15.5, 10.94, 10.5, 12.0, 5.06, 13.12, 10.0, 12

In [85]:
scores=pd.DataFrame()
scores['Name']=new_array[0]['Name']
for i in range(len(Parameters)):
    scores[Parameters[i]]=new_array[i+1][Parameters[i]]
scores.head()
scores.to_csv('Scores_Engineering_2021.csv',index=False)

# PDF DATA

In [14]:
import tabula
from csv import reader
import pandas as pd


def pdf_to_data(url):
    tabula.convert_into(url, "test.csv", output_format="csv", pages='all')
    all_tables=[]
    table=[]
    special_words=['Academic Year','(All programs\nof all years)','Ph.D (Student pursuing doctoral program till 2019-20)','No. of Ph.D students graduated (including Integrated Ph.D)','Financial Year','1. Do your institution buildings have Lifts/Ramps?','2. Do your institution have provision for walking aids, including wheelchairs and transportation from one building to another for handicapped students?','3. Do your institution buildings have specially designed toilets for handicapped students?','Number of faculty members entered']

    with open('test.csv', 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            for word in special_words:
                if(word in row):
                    if(len(table)>0):
                        all_tables.append(table)
                        table=[]
            updated_row=[]
            for word in row:
                    updated_word=word.replace("\n"," ")
                    updated_row.append(updated_word)
            table.append(updated_row)
        
    if(len(table)>0):
        all_tables.append(table)
        table=[]
    
    arr_df=[]
    for i in range(len(all_tables)):
        df = pd.DataFrame(data = all_tables[i])
        df.columns = df.iloc[0] 
        df = df[1:]
        arr_df.append(df)
#         display(df)
    return arr_df

In [15]:
Engineering_2021_pdf={'Name':[],'Data':[]}

for i in range(len(link)):
    print(i)
    Engineering_2021_pdf['Name'].append(link.loc[i,'Name'])
    ret=pdf_to_data(link.loc[i,'Pdf'])
    Engineering_2021_pdf['Data'].append(ret)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [16]:
display(len(Engineering_2021_pdf['Data'][0][0]))

4

In [17]:
# Name UG_total PG_Total PHd_full phd_part Faculty
Data=pd.DataFrame()
Data['Name']=Engineering_2021_pdf['Name']

In [18]:
# UG_INTAKE(NT)
UG=[]
for i in range(len(Engineering_2021_pdf['Data'])):
    total=0
    for k in range(len((Engineering_2021_pdf['Data'][i][0]))):
        if(Engineering_2021_pdf['Data'][i][0].iloc[k,0].find('UG')!=-1):
            for j in range(1,len(Engineering_2021_pdf['Data'][i][0].iloc[0])):
                if(Engineering_2021_pdf['Data'][i][0].iloc[k,j]!='-'):
                    total =total+int(Engineering_2021_pdf['Data'][i][0].iloc[k,j])
    UG.append(total)
print(len(UG))
Data['UG Intake']=UG
# Data.drop('UG')

200


In [19]:
# PG_INTAKE(NT)
PG=[]
for i in range(len(Engineering_2021_pdf['Data'])):
    total=0
    for k in range(len((Engineering_2021_pdf['Data'][i][0]))):
        if(Engineering_2021_pdf['Data'][i][0].iloc[k,0].find('PG')!=-1):
            for j in range(1,len(Engineering_2021_pdf['Data'][i][0].iloc[0])):
                if(Engineering_2021_pdf['Data'][i][0].iloc[k,j]!='-'):
                    total =total+int(Engineering_2021_pdf['Data'][i][0].iloc[k,j])
    PG.append(total)
print(len(PG))
Data['PG Intake']=PG

200


In [20]:
Data.head()

,Name,UG Intake,PG Intake
0,Indian Institute of Technology Madras,3813,1573
1,Indian Institute of Technology Delhi,3772,2214
2,Indian Institute of Technology Bombay,3744,1543
3,Indian Institute of Technology Kanpur,3471,1895
4,Indian Institute of Technology Kharagpur,2773,5946


In [21]:
N_T=[]
for i in range(200):
    tot=UG[i]+PG[i]
    N_T.append(tot)
Data['N_T']=N_T
Data.head()

,Name,UG Intake,PG Intake,N_T
0,Indian Institute of Technology Madras,3813,1573,5386
1,Indian Institute of Technology Delhi,3772,2214,5986
2,Indian Institute of Technology Bombay,3744,1543,5287
3,Indian Institute of Technology Kanpur,3471,1895,5366
4,Indian Institute of Technology Kharagpur,2773,5946,8719


In [22]:
Faculty=[]
for i in range(len(Engineering_2021_pdf['Data'])):
#     print(i)
    if(i==19):
        Faculty.append(388)
    elif(i==76):
        Faculty.append(354)
    elif(i==86):
        Faculty.append(79)
    else:        
        Faculty.append(int(Engineering_2021_pdf['Data'][i][-1].columns.values[1]))
Data['Faculty']=Faculty
Data.head()

,Name,UG Intake,PG Intake,N_T,Faculty
0,Indian Institute of Technology Madras,3813,1573,5386,642
1,Indian Institute of Technology Delhi,3772,2214,5986,667
2,Indian Institute of Technology Bombay,3744,1543,5287,592
3,Indian Institute of Technology Kanpur,3471,1895,5366,481
4,Indian Institute of Technology Kharagpur,2773,5946,8719,794


In [23]:
(Engineering_2021_pdf['Data'][0][6].columns[0])
# (Engineering_2021_pdf['Data'][0])

'Ph.D (Student pursuing doctoral program till 2019-20)'

In [24]:
Full_Time=[]
Part_Time=[]
for i in range(len(Engineering_2021_pdf['Data'])):
    for k in range(len(Engineering_2021_pdf['Data'][i])):
        if((Engineering_2021_pdf['Data'][i][k].columns[0])=='Ph.D (Student pursuing doctoral program till 2019-20)'):
            for j in range(len(Engineering_2021_pdf['Data'][i][k])):
                if(Engineering_2021_pdf['Data'][i][k].iloc[j][0]=='Full Time'):
                    Full_Time.append(int(Engineering_2021_pdf['Data'][i][k].iloc[j][1]))
                if(Engineering_2021_pdf['Data'][i][k].iloc[j][0]=='Part Time'):
                    Part_Time.append(int(Engineering_2021_pdf['Data'][i][k].iloc[j][1]))
Data['Full Time']=Full_Time
Data['Part Time']=Part_Time
Data.head()

,Name,UG Intake,PG Intake,N_T,Faculty,Full Time,Part Time
0,Indian Institute of Technology Madras,3813,1573,5386,642,2112,39
1,Indian Institute of Technology Delhi,3772,2214,5986,667,2349,570
2,Indian Institute of Technology Bombay,3744,1543,5287,592,1879,308
3,Indian Institute of Technology Kanpur,3471,1895,5366,481,1742,26
4,Indian Institute of Technology Kharagpur,2773,5946,8719,794,2963,0


In [25]:
# Data.to_csv('Data_Engineering_2021.csv',index=False)

In [27]:
# UG_INTAKE(NT)
UG=[]
PG=[]
for i in range(len(Engineering_2021_pdf['Data'])):
    ug=0
    pg=0
    for k in range(len((Engineering_2021_pdf['Data'][i][1]))):
        if(Engineering_2021_pdf['Data'][i][1].iloc[k,0].find('UG')!=-1):
            ug=ug+int(Engineering_2021_pdf['Data'][i][1].iloc[k,3])
        if(Engineering_2021_pdf['Data'][i][1].iloc[k,0].find('PG')!=-1):
            pg=pg+int(Engineering_2021_pdf['Data'][i][1].iloc[k,3])   
    UG.append(ug)
    PG.append(pg)
print(len(UG))
Data['UG Enrolled']=UG
Data['PG Enrolled']=PG
Data.head()

200


,Name,UG Intake,PG Intake,N_T,Faculty,Full Time,Part Time,UG Enrolled,PG Enrolled
0,Indian Institute of Technology Madras,3813,1573,5386,642,2112,39,3935,1914
1,Indian Institute of Technology Delhi,3772,2214,5986,667,2349,570,3988,2177
2,Indian Institute of Technology Bombay,3744,1543,5287,592,1879,308,4179,1738
3,Indian Institute of Technology Kanpur,3471,1895,5366,481,1742,26,3507,1759
4,Indian Institute of Technology Kharagpur,2773,5946,8719,794,2963,0,2519,6160


In [28]:
Data.to_csv('Data_Engineering_2021.csv',index=False)